# FMRI DATA ANALYSIS - NARPS DATASET TEAM T45A

This notebook reproduce the analysis made by team **T54A** for the NARPS experiment (Botvinik-Nezer & al, 2020). 

**To use this notebook :** 
- Download fMRIprep datas available [**here**](https://openneuro.org/datasets/ds001734/versions/1.0.4) or in [**datalad**](http://datasets.datalad.org/?dir=/workshops/nih-2017/ds001734).
- Modify the paths:
    - **exp_dir**: directory where the ds001734-download repository is stored
    - **result_dir**: directory where the intermediate and final repositories will be stored
    - **working_dir**: name of the directory where intermediate results will be stored
    - **output_dir**: name of the directory where final results will be stored

Botvinik-Nezer, R., Holzmeister, F., Camerer, C.F., Dreber, A., Huber, J., Johannesson, M., Kirchler, M., Iwanir, R., Mumford, J.A., ..., Nichols, T.E., Poldrack, R.A., Schonberg, T. (2020). Variability in the analysis of a single neuroimaging dataset by many teams. Nature. https://doi.org/10.1038/s41586-020-2314-9.

In [ ]:
from os.path import join as opj
import os
import json
from lib import pipelines_T54A

In [ ]:
team_ID = "T54A"

# Important directories
## exp_dir : where the data are stored (where the ds001734 directory is stored)
exp_dir = '/data/NARPS/ds001734-download'

## result_dir : where results are stored 
result_dir = '/local/egermani/analytic_variability_fmri/data/derived/reproduced'
data_dir = '/local/egermani/analytic_variability_fmri/data/derived/original'

## working_dir : where the intermediate outputs will be store
working_dir = f"NARPS-{team_ID}-reproduced/intermediate_results"

## output_dir : where the final results will be store
output_dir = f"NARPS-{team_ID}-reproduced"

## Single subject - Single run analysis

### In/out Nodes + Important values

In [ ]:
subject_list = ["001", "002", "003", "006", "008", "009", "010", "011",  "014", "015",  "017", "019", "020",
                "021", "022", "024", "025", "026", "027", "029", "032", "033", "035", "036", "037", "038", 
                "039", "040", "041", "043", "044", "045", "046", "047", "049", "050", "051", "052", "053", 
                "054", "055", "056", "057", "058", "059", "060", "061", "062", "063", "064", "066", "067", 
                 "069", "070", "071", "072", "074", "075", "076", "077", "079", "080", "081", 
                "082", "083", "084", "085", "087", "089", "090", "092", "093", "094", "095", "096", 
                "098", "102", "103", "104", "105", "106", "107", "108", "109", "110", "112", 
                "113", "114", "115", "117", "118", "119", "120", "121", "123", "124"]

In [ ]:
contrast_list = ['ploss', 'pgain']
n_sub = len(subject_list)

In [ ]:
#subject_list = ['001']
run_list = ['01', '02', '03', '04']
#TR
with open(opj(exp_dir, 'task-MGT_bold.json'), 'rt') as fp:
    task_info = json.load(fp)
TR = task_info['RepetitionTime']

#FWHM to smooth
fwhm = 4
l1_analysis= pipelines_T54A.get_l1_analysis(subject_list, run_list, 
                                            TR, fwhm, exp_dir, output_dir, 
                                            working_dir, result_dir)

In [ ]:
l1_analysis.run('MultiProc', plugin_args={'n_procs': 4})

In [ ]:
contrast_list = ['1', '2']
l2_analysis = pipelines_T54A.get_l2_analysis(subject_list, contrast_list, run_list, exp_dir, 
                                             output_dir, working_dir, result_dir, data_dir)

In [ ]:
l2_analysis.run('MultiProc', plugin_args={'n_procs': 4})

## Third level analysis - Group level 

In [ ]:
l3_analysis_equalRange = pipelines_T54A.get_group_workflow(subject_list, n_sub, contrast_list, 'equalRange', exp_dir, output_dir, 
                       working_dir, result_dir, data_dir)

l3_analysis_equalIndiff = pipelines_T54A.get_group_workflow(subject_list, n_sub, contrast_list, 'equalIndifference', 
                       exp_dir, output_dir, 
                       working_dir, result_dir, data_dir)

l3_analysis_groupComp = pipelines_T54A.get_group_workflow(subject_list, n_sub, contrast_list, "groupComp", exp_dir, 
                                                         output_dir, working_dir, result_dir, data_dir)

In [ ]:
l3_analysis_equalRange.run()

In [ ]:
l3_analysis_equalIndiff.run()

In [ ]:
l3_analysis_groupComp.run('MultiProc', plugin_args={'n_procs': 4})

In [ ]:
pipelines_T54A.reorganize_results(result_dir, output_dir, n_sub, team_ID)